In [71]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
from regression_dsets_large_names import regression_dsets_large_names
import numpy as np
import matplotlib.pyplot as plt
import pmlb
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data
import fit
from process_results import process_results, aggregate_results, best_ridge

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load results from a directory

In [58]:
!python process_results.py

100%|█████████████████████████████████████████████| 1/1 [00:16<00:00, 16.47s/it]


In [67]:
save_dir = 'results'
parent_dir = '/scratch/users/vision/yu_dl/raaz.rsk/double_descent/df'
folders = os.listdir(parent_dir)
print(folders)

['basic']


# linear plots

In [ ]:
for folder in folders:
    out_dir = oj(parent_dir, folder)
    df = pd.read_pickle(oj(out_dir, 'processed.pkl'))
    row_best_ridge = best_ridge(df) # contains df1, df2, df3
    len_ridge = row_best_ridge.df1.size
    
    
    R, C = 2, 3
    SIZE = 5
    plt.figure(figsize=(C * SIZE, R * SIZE))
    plt.subplot(R, C, 1)
    plt.axhline(df.mse_zero.values[0], lw=4, color='gray', alpha=0.4, label='trivial')
    for name, curve in tqdm(df.iterrows()):
    #     print(name, curve)
        model_type = name[3]
        reg_param = name[4]
        l = str(model_type) + ' ' + str(reg_param)
        if model_type == 'ols':
            ls = '--'
        elif model_type == 'ridge':
            ls = '-'
        elif model_type == 'linear_sta':
            ls = '-.'
        elif model_type == 'lasso':
            ls = ':'


        kwargs = {'label': l, 'alpha': 0.5, 'lw': 2, 'linestyle': ls}

        plt.subplot(R, C, 1)
    #     plt.plot(curve.ratio, curve.mse_test, 
        plt.plot(curve.ratio, curve.mse_noiseless, **kwargs) #np.square(curve.bias) + curve['var'], **kwargs)
        plt.ylabel('test mse')
        plt.xlabel('p / n')


        plt.subplot(R, C, 2)
        plt.plot(curve.ratio, curve.mse_train, **kwargs)
        plt.ylabel('train mse')
        plt.xlabel('p / n')
        
        
        
        plt.subplot(R, C, 4)
        plt.plot(row_best_ridge.df1, curve.mse_noiseless[:len_ridge], **kwargs)
        plt.ylabel('test mse')
        plt.xlabel('df1: $tr(S S^T)$')
        
        plt.subplot(R, C, 5)
        plt.plot(row_best_ridge.df2, curve.mse_noiseless[:len_ridge], **kwargs)
        plt.ylabel('test mse')
        plt.xlabel('df2: $tr(2 * S - S^T S)$')
        
        plt.subplot(R, C, 6)
        plt.plot(row_best_ridge.df3, curve.mse_noiseless[:len_ridge], **kwargs)
        plt.ylabel('test mse')
        plt.xlabel('df3: $tr(S)$')
        
        
        '''
        plt.subplot(R, C, 3)
        plt.plot(curve.ratio, np.square(curve.bias), **kwargs)
        plt.ylabel('bias')
        
        plt.subplot(R, C, 4)
        plt.plot(curve.ratio, curve['var'], **kwargs)
        plt.ylabel('var')
        '''
        
        '''
        # wnorm also recorded
        plt.subplot(R, C, 3)
        plt.plot(curve.ratio, curve['wnorm'], **kwargs)
        plt.ylabel('wnorm')
        plt.xlabel('p / n')
        
        plt.subplot(R, C, 4)
        plt.plot(curve.ratio, curve['num_nonzero'], **kwargs)
        plt.ylabel('num_nonzero')
        plt.xlabel('p / n')
        '''

    for i in range(1, R * C + 1):
        plt.subplot(R, C, i)
        plt.xscale('log')
        plt.yscale('log')

    #     s = f'{model_type}_{dset}_{dset_name}+p={num_features}+lambda={reg_param}+noise_mult={noise_mult}'
    plt.subplot(R, C, 1)
    plt.legend()
    
    plt.subplot(R, C, 3)
    ratios = row_best_ridge.ratio
    n = 500 / ratios
    p = 0 * ratios + 500
    plt.plot(ratios, np.minimum(n, p), label='min(n, p)', alpha=0.5)
    plt.plot(ratios, row_best_ridge.df1, label='df1')
    plt.plot(ratios, row_best_ridge.df2, label='df2')
    plt.plot(ratios, row_best_ridge.df3, label='df3')
    plt.xlabel('p / n')
    plt.legend()
    
    
    s = folder
    plt.suptitle(s)
    plt.tight_layout()
    # plt.show()
    plt.savefig(oj(save_dir, s + '.pdf'))
    plt.show()
    plt.close()